In [1]:
import os
os.environ["KERAS_BACKEND"] ='tensorflow'
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
from tqdm.auto import tqdm
tqdm.pandas()
import keras 
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
from keras import layers, Model, Input
from keras_hub.src.models.llama.llama_decoder import LlamaTransformerDecoder
from keras_hub.src.models.llama.llama_layernorm import LlamaLayerNorm
tqdm.pandas()

/home/mhossai5/.conda/envs/antibody_dl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026-01-04 06:50:47.970231: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767531048.163085    8582 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767531048.172666    8582 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767531048.455051    8582 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more 

# Data Loading

In [2]:
df = pd.read_csv("../ppi/data/LR_PPI.csv")
max_seq = 1500
df = df[(df['Protein_1_Seq'].str.len()<=max_seq) & (df['Protein_2_Seq'].str.len()<=max_seq)]# .groupby('label',group_keys =False).apply(lambda x: x.sample(4000,replace=False))
print(df.shape)
df.head()

(65849, 6)


,Index,Protein_1_ID,Protein_2_ID,Protein_1_Seq,Protein_2_Seq,label
0,1,NP_663777.1,NP_001233.1,MESSKKMDSPGALQTNPPLKLHTDRSAGTPVFVPEQGGYKEKFVKT...,MARPHPWWLCVLGTLVGLSATPAPKSCPERHYWAQGKLCCQMCEPG...,0
2,3,NP_003630.1,NP_001073594.1,MNRHLWKSQLCEMVQPSGGPAADQDVLGEESPLGKPAMLHLPSEQG...,MEGGRRARVVIESKRNFFLGAFPTPFPAEHVELGRLGDSETAMVPG...,0
3,4,NP_001001998.1,NP_067000.1,MAPPSTREPRVLSATSATKSDGEMVLPGFPDADSFVKFALGSVVAV...,MLFYSFFKSLVGKDVVVELKNDLSICGTLHSVDQYLNIKLTDISVT...,0
5,6,NP_001033.1,NP_002037.2,MPSGFQQIGSEDGEPPQQRVTGTLVLAVFSAVLGSLQFGYNIGVIN...,MGKVKVGVNGFGRIGRLVTRAAFNSGKVDIVAINDPFIDLNYMVYM...,0
6,7,NP_003001.1,NP_006292.2,MAAPSPSGGGGSGGGSGSGTPGPVGSPAPGHPAVSSMQGKRKALKL...,MACLHETRTPSPSFGGFVSTLSEASMRKLDPDTSDCTPEKDLTPTH...,0


In [3]:
import h5py
file_path = "../ppi/data/LR_PPI-ProtT5-Full.h5"

loaded_data = {}
with h5py.File(file_path, 'r') as hf:
    for seq in hf.keys():
        loaded_data[seq] = hf[seq][:]

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import Sequence

def process_sequence_tf(x_emb, max_len=512, pad_value=0.0):
    seq_len = x_emb.shape[0]
    
    if seq_len > max_len:
        # truncate
        x_emb = tf.convert_to_tensor(x_emb[:max_len])
        mask = tf.ones([max_len], dtype=tf.float32)
    else:
        # pad
        pad_len = max_len - seq_len
        paddings = [[0, pad_len], [0, 0]]
        x_emb = tf.pad(x_emb, paddings, constant_values=pad_value)
        mask = tf.pad(tf.ones([seq_len], dtype=tf.float32), [[0, pad_len]], constant_values=0.0)
    
    return x_emb, mask

# -----------------------------
# Keras Sequence Loader
# -----------------------------
class DataSequenceLoader(Sequence):
    def __init__(self, df, batch_size=32, shuffle=True, max_len=512, pad_value=0.0):
        self.prot1_emb = df["Protein_1_Seq"].values
        self.prot2_emb = df["Protein_2_Seq"].values
        self.labels = df["label"].values
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.max_len = max_len
        self.pad_value = pad_value
        self.indices = np.arange(len(df))
        if shuffle:
            np.random.shuffle(self.indices)

    def __len__(self):
        return int(np.ceil(len(self.indices) / self.batch_size))

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

    def __getitem__(self, idx):
        batch_idx = self.indices[idx*self.batch_size:(idx+1)*self.batch_size]

        x1_list, x2_list, m1_list, m2_list = [], [], [], []
        labels_list = []

        for i in batch_idx:
            emb1 = loaded_data[self.prot1_emb[i]].reshape(-1, 1024)
            emb2 = loaded_data[self.prot2_emb[i]].reshape(-1, 1024)

            # ---- process embeddings + mask ----
            x1_pad, mask1 = process_sequence_tf(emb1, max_len=self.max_len, pad_value=self.pad_value)
            x2_pad, mask2 = process_sequence_tf(emb2, max_len=self.max_len, pad_value=self.pad_value)

            x1_list.append(x1_pad.numpy())
            x2_list.append(x2_pad.numpy())
            m1_list.append(mask1.numpy())
            m2_list.append(mask2.numpy())
            labels_list.append(self.labels[i])

        # Convert lists to arrays for batch
        x1_batch = np.stack(x1_list, axis=0)
        x2_batch = np.stack(x2_list, axis=0)
        m1_batch = np.stack(m1_list, axis=0)
        m2_batch = np.stack(m2_list, axis=0)
        labels_batch = np.array(labels_list)

        return (x1_batch, x2_batch, m1_batch, m2_batch), labels_batch


In [5]:
# -------------------------------------------------------------------
# Hybrid Pooling Layer (Max + Avg)
# -------------------------------------------------------------------
class HybridPooling(layers.Layer):
    def call(self, x):
        max_pooled = keras.ops.max(x, axis=1)
        avg_pooled = keras.ops.mean(x, axis=1)
        return keras.ops.concatenate([max_pooled, avg_pooled], axis=-1)


# -------------------------------------------------------------------
# Conv Block (Conv1D → ReLU → Dropout → MaxPool)
# -------------------------------------------------------------------
def conv_block(x, filters =  100, kernel_sz =20, stride =10, dropout = 0.5):
    x = layers.Conv1D(filters, kernel_sz, strides=stride)(x)
    x = layers.ReLU()(x)
    x = layers.Dropout(dropout)(x)
    x = layers.MaxPooling1D(pool_size=3, strides=1, padding="same")(x)
    return x


# -------------------------------------------------------------------
# Cross-attention (Query=X1, Key=X2, Value=X2)
# -------------------------------------------------------------------
def cross_attention_block(query, key, value, mask, num_heads =4, key_dim =32):
    attn = layers.MultiHeadAttention(
        num_heads=num_heads, 
        key_dim=key_dim,
        dropout=0.0,
        kernel_initializer="glorot_uniform",
        use_bias=True,
        # flash_attention=None,  # if GPU supports
    )(query, key, value, key_mask=mask, value_mask=mask)
    return attn


# -------------------------------------------------------------------
# LLaMA Self-Attention Block
# -------------------------------------------------------------------
def llama_self_attention(x, mask, hidden_dim =100, num_heads = 4):
    # single LLaMA decoder layer
    llama = LlamaTransformerDecoder(
        intermediate_dim=hidden_dim * 4,
        # num_heads=num_heads,
        num_query_heads=8,
        num_key_value_heads=2,
        dropout=0.0,
        layer_norm_epsilon=1e-5,
        activation="silu"
    )
    return llama(x, decoder_padding_mask=mask)

# -------------------------------------------------------------------
# Functional API Model (PyTorch → Keras Conversion)
# -------------------------------------------------------------------
def build_model(
    input_dim=1024,
    conv_out=100,
    kernel_sz=20,
    stride=10,
    heads=4,
    d_dim=32,
    drop_pool=0.5,
    drop_linear=0.3
):
    # Inputs
    inp1 = Input((None, input_dim))
    inp2 = Input((None, input_dim))
    mask1 = Input((None, ))
    mask2 = Input((None, ))

    # ---------------------------------------------------
    # 1) Convolutional features
    # ---------------------------------------------------
    x1 = layers.Dense(100, use_bias=False,name='stem_conv1')(inp1)
    x1 = layers.BatchNormalization(momentum=0.95,name='stem_bn1')(x1)
    
    x2 = layers.Dense(100, use_bias=False,name='stem_conv2')(inp2)
    x2 = layers.BatchNormalization(momentum=0.95,name='stem_bn2')(x2)
    
    # ---------------------------------------------------
    # 2) Self Attention using LLaMA blocks
    # ---------------------------------------------------
    s1 = llama_self_attention(x1, mask1, conv_out, heads)
    s2 = llama_self_attention(x2, mask2, conv_out, heads)

    # ---------------------------------------------------
    # 3) Cross Attention (1→2 and 2→1)
    # ---------------------------------------------------
    c1 = cross_attention_block(x1, x2, x2, mask2, heads, d_dim)
    c2 = cross_attention_block(x2, x1, x1, mask1, heads, d_dim)

    # Add residual (same as PyTorch + skip)
    sc1 = layers.Add()([s1, c1])
    sc2 = layers.Add()([s2, c2])

    sc1 = layers.Dropout(drop_pool)(sc1)
    sc2 = layers.Dropout(drop_pool)(sc2)

    # ---------------------------------------------------
    # 4) Hybrid Pooling (max + mean)
    # ---------------------------------------------------
    h1 = HybridPooling()(sc1)
    h2 = HybridPooling()(sc2)

    merged = layers.Concatenate()([h1, h2])

    # ---------------------------------------------------
    # 5) MLP Head
    # ---------------------------------------------------
    x = layers.Dense(256, activation="relu")(merged)
    x = layers.Dropout(drop_linear)(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(drop_linear)(x)
    out = layers.Dense(1, activation="sigmoid")(x)

    return Model(inputs=[inp1, inp2, mask1, mask2], outputs=out)




In [6]:
# !conda install wandb -y

In [7]:
import os
import time
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    accuracy_score, f1_score, recall_score, precision_score,
    roc_auc_score, average_precision_score, confusion_matrix
)
import wandb
from wandb.integration.keras import WandbMetricsLogger

In [8]:
# =========================================================
# DEFAULT CONFIG (Logged Automatically to W&B)
# =========================================================
TIMESTAMP = time.strftime("%Y%m%d-%H%M%S")

CONFIG = {
    "batch_size": 4,
    "n_splits": 5,
    "epochs": 10,
    "learning_rate": 1e-4,
    "optimizer": "Adam",
    "loss": "binary_crossentropy",
    "architecture": "ProtT5",
    "dataset": "Human-LR_PPI",
    "task": "Prot-Prot Classification",
    "random_state": 42,
    "max_len":512,
}

PROJECT_NAME = f"{CONFIG['dataset']}-RELAAI-{CONFIG['architecture']}-{TIMESTAMP}"
OUT_PATH = os.path.join("weights",PROJECT_NAME)
os.makedirs(os.path.join(OUT_PATH, "logs"), exist_ok=True)
os.makedirs(os.path.join(OUT_PATH, "weights"), exist_ok=True)
OUT_PATH

'weights/Human-LR_PPI-RELAAI-ProtT5-20260104-065453'

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    accuracy_score, f1_score, recall_score, precision_score,
    roc_auc_score, matthews_corrcoef, confusion_matrix,
    average_precision_score
)
CONFIG['n_splits'] =5
# =============================================================
#  K-FOLD CROSS VALIDATION SETTINGS
# =============================================================
skf = StratifiedKFold(n_splits=CONFIG['n_splits'], shuffle=True, random_state=42)

all_metrics = []

# =============================================================
#  MAIN LOOP
# =============================================================
for fold,(train_idx, valid_idx) in enumerate(skf.split(df, df["label"]),1):
    print(f"\n==========================")
    print(f" Fold {fold} / {CONFIG['n_splits']}")
    print(f"==========================")
    run = wandb.init(
            project=PROJECT_NAME,
            name=f"fold_{fold}",
            group="KFold-CV",
            config={**CONFIG, "fold": fold},
            reinit=True
        )
    train_df = df.iloc[train_idx].reset_index(drop=True)
    valid_df = df.iloc[valid_idx].reset_index(drop=True)

    # ---------------------------------------
    # Loaders
    # ---------------------------------------
    train_loader = DataSequenceLoader(
        train_df, 
        batch_size=CONFIG["batch_size"],
        max_len = CONFIG['max_len'], 
        shuffle=True)
    valid_loader = DataSequenceLoader(
        valid_df, 
        batch_size=CONFIG["batch_size"],
        max_len = CONFIG['max_len'], 
        shuffle=False)

    # ---------------------------------------
    # Build a FRESH MODEL per fold
    # ---------------------------------------
    model = build_model()
    model.compile(
        optimizer=tf.keras.optimizers.Adam(CONFIG["learning_rate"]),
        loss=CONFIG["loss"],
        metrics=[
            "accuracy",
            tf.keras.metrics.AUC(name="auc")
        ]
    )
    # -----------------------------------------------------
    # Callbacks
    # -----------------------------------------------------
    tb_callback = tf.keras.callbacks.TensorBoard(
        log_dir=os.path.join(OUT_PATH, "logs", f"fold_{fold}")
    )

    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(
            OUT_PATH, "weights",
            f"weights_fold{fold}-best.weights.h5"
        ),
        save_weights_only=True,
        save_best_only=True,
        monitor="val_loss"
    )
    # ---------------------------------------
    # Train
    # ---------------------------------------
    history = model.fit(
        train_loader,
        validation_data=valid_loader,
        epochs=CONFIG["epochs"],
        callbacks=[
            tb_callback,
            checkpoint_cb,
            WandbMetricsLogger(log_freq="epoch")
        ],
        verbose=1
    )

    model.save_weights(
        os.path.join(
            OUT_PATH,
            "weights",
            f"weights_fold{fold}-last.weights.h5"
        )
    )

    # -----------------------------------------------------
    # Evaluation
    # -----------------------------------------------------
    y_pred_prob = model.predict(valid_loader).ravel()
    y_pred = (y_pred_prob > 0.5).astype(int)
    y_true = valid_df["label"].values

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0

    metrics_dict = {
        "fold": fold,
        "accuracy": accuracy_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred),
        "roc_auc": roc_auc_score(y_true, y_pred_prob),
        "aupr": average_precision_score(y_true, y_pred_prob),
        "specificity": specificity
    }

    # Log fold metrics to W&B
    wandb.log(metrics_dict)

    print(metrics_dict)
    all_metrics.append(metrics_dict)

    run.finish()

# =========================================================
# Save All Metrics
# =========================================================
metrics_df = pd.DataFrame(all_metrics)
avg_row = metrics_df.mean(numeric_only=True)
metrics_df = pd.concat(
    [metrics_df, avg_row.to_frame().T],
    ignore_index=True
)
metrics_df.loc[metrics_df.index[-1], "fold"] = "Average"

metrics_df.to_csv(
    os.path.join(
        OUT_PATH,
        f"{PROJECT_NAME}-kfold_classification_metrics.csv"
    ),
    index=False
)

print("\nAll fold metrics saved.")
metrics_df


 Fold 1 / 5


wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hossainstudy7 (hossainstudy7-freelancer) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


I0000 00:00:1767531309.304466    8582 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:04:00.0, compute capability: 6.0
/home/mhossai5/.conda/envs/antibody_dl/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


/home/mhossai5/.conda/envs/antibody_dl/lib/python3.10/site-packages/keras/src/callbacks/tensorboard.py:680: UserWarning: Model failed to serialize as JSON. Ignoring... 'LlamaTransformerDecoder' object has no attribute 'rope_scaling_factor'
  warnings.warn(f"Model failed to serialize as JSON. Ignoring... {exc}")
I0000 00:00:1767531322.988159    9909 service.cc:152] XLA service 0x7ffe54001500 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1767531322.988206    9909 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1767531324.627534    9909 cuda_dnn.cc:529] Loaded cuDNN version 90500
I0000 00:00:1767531331.510152    9909 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...


 Fold 3 / 5


Epoch 1/10


/home/mhossai5/.conda/envs/antibody_dl/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
/home/mhossai5/.conda/envs/antibody_dl/lib/python3.10/site-packages/keras/src/callbacks/tensorboard.py:680: UserWarning: Model failed to serialize as JSON. Ignoring... 'LlamaTransformerDecoder' object has no attribute 'rope_scaling_factor'
  warnings.warn(f"Model failed to serialize as JSON. Ignoring... {exc}")


13170/13170 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.8083 - auc: 0.8756 - loss: 0.4429

13170/13170 ━━━━━━━━━━━━━━━━━━━━ 486s 36ms/step - accuracy: 0.8949 - auc: 0.9584 - loss: 0.2639 - val_accuracy: 0.9766 - val_auc: 0.9941 - val_loss: 0.1356
Epoch 2/10
13170/13170 ━━━━━━━━━━━━━━━━━━━━ 455s 35ms/step - accuracy: 0.9689 - auc: 0.9917 - loss: 0.1021 - val_accuracy: 0.9835 - val_auc: 0.9948 - val_loss: 0.0722
Epoch 3/10
13170/13170 ━━━━━━━━━━━━━━━━━━━━ 456s 35ms/step - accuracy: 0.9778 - auc: 0.9945 - loss: 0.0762 - val_accuracy: 0.9819 - val_auc: 0.9954 - val_loss: 0.0859
Epoch 4/10
13170/13170 ━━━━━━━━━━━━━━━━━━━━ 454s 34ms/step - accuracy: 0.9809 - auc: 0.9955 - loss: 0.0654 - val_accuracy: 0.9767 - val_auc: 0.9959 - val_loss: 0.1066
Epoch 5/10
13170/13170 ━━━━━━━━━━━━━━━━━━━━ 453s 34ms/step - accuracy: 0.9835 - auc: 0.9965 - loss: 0.0572 - val_accuracy: 0.9814 - val_auc: 0.9963 - val_loss: 0.0754
Epoch 6/10
13170/13170 ━━━━━━━━━━━━━━━━━━━━ 452s 34ms/step - accuracy: 0.9845 - auc: 0.9965 - loss: 0.0538 - val_accuracy: 0.9853 - val_auc: 0.9961 - val_loss: 0.0623
Epoch 7/1

In [ ]:
# =============================================================
#  EVALUATION (POST-TRAINING)
# =============================================================
from sklearn.metrics import (
    accuracy_score, f1_score, recall_score, precision_score,
    roc_auc_score, matthews_corrcoef,
    confusion_matrix, average_precision_score
)

all_metrics = []
skf = StratifiedKFold(n_splits=CONFIG['n_splits'], shuffle=True, random_state=42)

for fold, (_, valid_idx) in enumerate(
        skf.split(df, df["Interaction"]), 1):

    print(f"\nEvaluating Fold {fold}")

    valid_df = df.iloc[valid_idx].reset_index(drop=True)

    valid_loader = DataSequenceLoader(
        valid_df,
        batch_size=CONFIG["batch_size"],
        max_len=CONFIG["max_len"],
        shuffle=False
    )

    # Rebuild model & load best weights
    model = build_model()
    model.load_weights(
        os.path.join(
            OUT_PATH, "weights",
            f"weights_fold{fold}-best.weights.h5"
        )
    )

    # Prediction
    y_prob = model.predict(valid_loader).ravel()
    y_pred = (y_prob >= 0.5).astype(int)
    y_true = valid_df["label"].values

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    metrics_dict = {
        "fold": fold,
        "accuracy": accuracy_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),     # sensitivity
        "precision": precision_score(y_true, y_pred),
        "mcc": matthews_corrcoef(y_true, y_pred),
        "auc": roc_auc_score(y_true, y_prob),
        "prauc": average_precision_score(y_true, y_prob),
        "specificity": tn / (tn + fp) if (tn + fp) > 0 else 0.0,
        "sensitivity": tp / (tp + fn) if (tp + fn) > 0 else 0.0
    }

    print(metrics_dict)
    all_metrics.append(metrics_dict)
metrics_df = pd.DataFrame(all_metrics)

avg_row = metrics_df.mean(numeric_only=True)
avg_row["fold"] = "Average"

metrics_df = pd.concat(
    [metrics_df, avg_row.to_frame().T],
    ignore_index=True
)

metrics_df.to_csv(
    os.path.join(
        OUT_PATH,
        f"{PROJECT_NAME}-kfold_evaluation_metrics.csv"
    ),
    index=False
)

print("\nEvaluation completed and saved.")
metrics_df
